In [36]:
import requests, bs4, selenium
from PyPDF2 import PdfFileReader
from selenium import webdriver
#link = ["https://ocw.mit.edu/courses/11-s940-development-planning-and-implementation-the-dialectic-of-theory-and-practice-fall-2015/", "https://ocw.mit.edu/courses/11-s955-the-sustainability-response-to-covid-19-january-iap-2021/", "https://ocw.mit.edu/courses/12-007-geobiology-spring-2013/", "https://ocw.mit.edu/courses/16-810-engineering-design-and-rapid-prototyping-january-iap-2007/"]

In [8]:
PATH = './driver/chromedriver.exe'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}
pref = 'https://ocw.mit.edu'

In [9]:
class CourseRes:
    formato=""
    typeR=""
    nome=""
    link=""
    durata=""

    def __init__(self, nome, link):
        self.nome=nome
        self.formato="pdf"
        self.typeR="presentazione"
        self.link=link

    def toString(self):
        return "{} {}.{} in {}".format(self.typeR, self.nome, self.formato, self.link)

class CourseInfo:
    nome=""
    idN=""
    link=""
    kywTit=""
    difficulty=""
    res=[] # type CourseRes
    min_age=0
    max_age=0
    desc=""
    discip=""
    pubblico=[]

    def __init__(self, link, name, idl, difficulty, topic, desc):
        self.discip=topic
        self.link=link
        self.nome=name
        self.idN=idl
        self.difficulty=difficulty
        self.desc=desc
        if "undergraduate" in difficulty.lower():
            self.pubblico = ['Middle School', 'High School']
            self.min_age=11
            self.max_age=18
        else:
            self.pubblico = ['College General Ed', 'Graduate School', 'Professional']
            self.min_age=19
            self.max_age=100


    #difficulty: graduate=medio alta, undergraduate=media
    #formato: lecture notes=pdf, assignments=pdf, 
    #durata: slidex2 = minuti {[0-30]...[120+]}, 
    #pubblico: undergrad=['Middle School', 'High School'], graduate=['College General Ed', 'Graduate School', 'Professional]
    #type: lecture notes=presentation, assignments=Quiz/Test, 
    #min_age: undergrad=11, grad=19, max_age: undergrad=18, grad=100

    def printInCSV(self, file):
        for attr in self.__dict__:
            item = self.__dict__[attr]
            if not type(item) is list:
                file.write("{};".format(item))
            else:
                if isinstance(item[0], CourseRes):
                    file.write("{};".format([r.toString() for r in item]))
                else:
                    file.write("{};".format(item))
        file.write("\n")

In [12]:
def datacollector (link):
    driver = webdriver.Chrome(PATH)
    driver.get(link)
    html = driver.page_source
    d = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "course-description"})[0].text.replace("\n", " ")
    lvl = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "course-info-level"})[0].text.replace(" ", "").replace("\n", "")
    tit = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "text-uppercase display-4 font-weight-bold m-0 text-white"})[0].text.replace("\n", "")
    idl = bs4.BeautifulSoup(html, "html.parser").find_all("ul", {"class" : "list-unstyled m-0"})[1].text.replace(" ", "").replace("\n", "")
    t = bs4.BeautifulSoup(html, "html.parser").find_all("table", {"id" : "course-info-table"})
    topic = ""
    for c in t:
        for a in c.find_all("ul", {"class" : "list-unstyled pb-2 m-0"}):
            for g in a.find_all("li"):
                topic += g.text.replace("\n", "").replace ("      >          ", "/")
    course = CourseInfo(link, tit, idl, lvl, topic, d)
    driver.quit()
    course.res = getCoursePDF(link+"pages/lecture-notes/") 
    return course

In [29]:
def getCoursePDF(link):
    courseRes=[]
    driver = webdriver.Chrome(PATH)
    driver.get(link)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "main-content"})
    for s1 in soup:
        for s in s1.find_all("td"):
            for e in s.find_all("a"):
                nlink=pref+e.attrs["href"]
                obj = CourseRes(s.text, nlink)
            
                response = requests.get(nlink)
                my_raw_data = response.content
            
                with open("my_pdf.pdf", 'wb') as my_data:
                    my_data.write(my_raw_data) 
            
                open_pdf_file = open("my_pdf.pdf", 'rb')
                read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
                if read_pdf.isEncrypted:
                    read_pdf.decrypt("")
                    obj.durata="circa {} minuti".format(read_pdf.getNumPages())
                    print(obj.durata)
                else:
                    obj.durata="circa {} minuti".format(read_pdf.getNumPages())
                    print(obj.durata)
                courseRes.append(obj)
                del obj
    driver.quit()
    return courseRes

ls = getCoursePDF('https://ocw.mit.edu/courses/16-810-engineering-design-and-rapid-prototyping-january-iap-2007/pages/lecture-notes/')
for i in ls:
    print(i.toString())

        

PdfReadError: EOF marker not found

In [32]:
f = open("res_dataset.csv", "a")
for i in link:
    course = datacollector(i)
    course.printInCSV(f)
    del course
f.close()

C:\Users\mirco\AppData\Local\Temp/ipykernel_4756/2613464865.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [9]:
driver = webdriver.Chrome(PATH)
driver.get('https://ocw.mit.edu/pdfjs/web/viewer.html?file=blob:https://ocw.mit.edu/b26e9740-2fa5-41c2-b977-19194e3f2876')

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_9480/1983120312.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [2]:
lst=[[1,2],[3,5,6]]
flat_1 = [x for l in lst for x in l]

In [3]:
flat_1

[1, 2, 3, 5, 6]

In [38]:
response = requests.get('https://ocw.mit.edu/pdfjs/web/viewer.html?file=blob%3Ahttps%3A%2F%2Focw.mit.edu%2Fa100be41-3106-448d-865c-c9760e7eec39')
my_raw_data = response.content
duration=""
            
with open("my_pdf.pdf", 'wb') as my_data:
    my_data.write(my_raw_data+b'%%EOF')
            
open_pdf_file = open("my_pdf.pdf", 'rb')
read_pdf = PdfFileReader(open_pdf_file)
if read_pdf.isEncrypted:
    read_pdf.decrypt("")
    duration="circa {} minuti".format(read_pdf.getNumPages())
    print(duration)
else:
    duration="circa {} minuti".format(read_pdf.getNumPages())
    print(duration)

PdfReadError: startxref not found